In [ ]:
using Pkg

function install_missing_package(package_name::String)
    if !haskey(Pkg.installed(), package_name)
        println("Package \""*package_name*"\" is missing.")
        println("Installing package...")
        Pkg.add(package_name)
    else
        println("Package \""*package_name*"\" is already installed.")
    end
     println("=======================================================")
end

install_missing_package("Eirene")
install_missing_package("DelimitedFiles")
install_missing_package("PyCall")
install_missing_package("Plots")

# Load packages
Notice 1: Eirene is precompiling at every restart of kernel

Notice 2: Some warnings may occur during the Eirene precompilation. This is beacuse of unfixed Eirene issues. Despite the warnings, library should work fine.

In [ ]:
using DelimitedFiles
using PyCall
using Plots
using Eirene

# Load file:

## a) from ".npy" file with numpy:
Notice: "SyllableEncoding.npy" is not included in the repository

In [ ]:
PyAny = py"""
        import numpy as np
        import matplotlib.pyplot as plt

        file_name = "SyllableEncoding.npy"

        py_symm_matrtix = np.load(file_name, allow_pickle=True, encoding="latin1")
        """
py_symmetric_matrtix = PyArray(py"py_symm_matrtix"o)
symmetric_matrix = Array(py_symmetric_matrtix)

## b) from ".txt" or ".csv" file with julia:

In [ ]:
file_name = "geometric_matrix.csv"
symmetric_matrix = readdlm( file_name, ',', Float64, '\n')

# Preprocessing

## Ordering of the matrix

In [ ]:
function get_ordered_matrix(input_matrix)
    mat_size = size(input_matrix,1)
    ordered_matrix = zeros(Int, mat_size, mat_size)

    # ====
    # Get all cartesian indices from input matrix
    matrix_indices = CartesianIndices((1:mat_size, 1:mat_size))
    # Filter out indices below diagonal
    matrix_indices = findall(x->x[1]>x[2], matrix_indices)

    # Count elements above diagonal
    repetition_number = Int(ceil((mat_size * (mat_size-1))/2))
   
    # Get all values which will be sorted
    sorted_values = input_matrix[matrix_indices]

    # Sort indices by values (starting with lowest)
    ordered_indices = sort!([1:repetition_number;],
                        by=i->(sorted_values[i],matrix_indices[i]))

    for k=1:repetition_number
        next_position = ordered_indices[k]
        matrix_index = matrix_indices[next_position]
        ordered_matrix[matrix_index] = k
        ordered_matrix[matrix_index[2], matrix_index[1]] = k
    end

    # ====
    # Check the ordering
    non_zero_input = findall(x->x!=0,input_matrix)
    min_orig = findmin(input_matrix[non_zero_input])[2]
    max_new = findall(x->x==1,ordered_matrix)[1]
    println("Check:")
    println("\tOriginal minimal value was at position: \t"*string(non_zero_input[min_orig]))
    println("\tAfter ordering the first index value is at position: \t"*string(max_new))
    return ordered_matrix
end

### Simple check of ordering

In [ ]:
a = [ 0 11 12;
     11  0 13;
     12 13  0];
get_ordered_matrix(a)

## Order input matrix

In [ ]:
las_index = 60
ordered_matrix = get_ordered_matrix(symmetric_matrix[1:las_index,1:las_index])

# Topological analysis

## Use Eirene

In [ ]:
maximal_dimension = 5

persistance_analysis = eirene(ordered_matrix,maxdim=maximal_dimension)

## Compute Betti curves

In [ ]:
minimal_betti = 1
maximal_betti = maximal_dimension

bettis  = Matrix{Float64}[]
for d =minimal_betti:(maximal_betti+1)
    result = betticurve(persistance_analysis, dim=d)
    push!(bettis, result)
end
# bettis

## Plot betti curves

In [ ]:
plot_ref = plot(title="");

for p = (minimal_betti):(maximal_betti)
    plot!(bettis[p][:,1], bettis[p][:,2], label="\\beta_"*string(p),)
end

plot_ref

## Plot heatmap of ordering matrix

In [ ]:
heatmap(ordered_matrix,  color=:lightrainbow)